# **02 - Sale Price Study**

## Objectives

*   Answer business requirement 1: 
    * The client is interested in discovering how the house attributes correlate with the sale price. Therefore, the client expects data visualisations of the correlated variables against the sale price to show that.

## Inputs

* outputs/datasets/collection/house_prices_records.csv

## Outputs

* generate code that answers business requirement 1 and can be used to build the Streamlit App






---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Load Data

In [ ]:
import pandas as pd
df = (pd.read_csv("outputs/datasets/collection/house_prices_records.csv")
    )
df.head(3)

---

# Data Exploration

Install Requirements

In [ ]:
%pip install ipywidgets==8.0.2 feature_engine plotly


In [ ]:
from pandas_profiling import ProfileReport
pandas_report = ProfileReport(df=df, minimal=True)
pandas_report.to_notebook_iframe()

---

# Correlation Study

We use `.corr()` for `spearman` and `pearson` methods, and investigate the top 10 correlations
* We know this command returns a pandas series and the first item is the correlation between Churn and Churn, which happens to be 1, so we exclude that with `[1:]`
* We sort values considering the absolute value, by setting `key=abs`

In [ ]:
corr_spearman = df.corr(method='spearman')['SalePrice'].sort_values(key=abs, ascending=False)[1:].head(10)
corr_spearman

We do the same for `pearson`

In [ ]:
corr_pearson = df.corr(method='pearson')['SalePrice'].sort_values(key=abs, ascending=False)[1:].head(10)
corr_pearson

We will consider the top correlation levels

In [ ]:
top_n = 5
set(corr_pearson[:top_n].index.to_list() + corr_spearman[:top_n].index.to_list())

Therefore we are studying at df the following variables. We will investigate if:
* A larger 1stFlrSF leads to a higher SalePrice.
* A larger GarageArea leads to a higher SalePrice.
* A larger GrLivArea leads to a higher SalePrice.
* How the OverallQual influences SalePrice.
* A larger TotalBsmntSF leads to a higher SalePrice.
* How the YearBuilt influences SalePrice.

In [ ]:
vars_to_study = ['1stFlrSF', 'GarageArea', 'GrLivArea', 'OverallQual', 'TotalBsmtSF', 'YearBuilt']
vars_to_study

# EDA on selected variables

In [ ]:
df_eda = df.filter(vars_to_study + ['SalePrice'])
df_eda.head(3)

## Variables Distribution by SalePrice

We plot the distribution (numerical and categorical) coloured by SalePrice

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')


def plot_categorical(df, col, target_var):

    plt.figure(figsize=(12, 5))
    sns.countplot(data=df, x=col, hue=target_var, order=df[col].value_counts().index)
    plt.xticks(rotation=90)
    plt.title(f"{col}", fontsize=20, y=1.05)
    plt.show()


def plot_numerical(df, col, target_var):
    plt.figure(figsize=(8, 5))
    sns.histplot(data=df, x=col, hue=target_var, kde=True, element="step")
    plt.title(f"{col}", fontsize=20, y=1.05)
    plt.show()


target_var = 'SalePrice'
for col in vars_to_study:
    if df_eda[col].dtype == 'object':
        plot_categorical(df_eda, col, target_var)
        print("\n\n")
    else:
        plot_numerical(df_eda, col, target_var)
        print("\n\n")


## Parallel Plot

Because the SalePrice has so many data pints, we create a separate DataFrame and transform `SalePrice` (numerical) into bins (categorical) for visualising at `parallel_categories()` plot


In [ ]:
from feature_engine.discretisation import ArbitraryDiscretiser
import numpy as np
saleprice_map = [-np.Inf, 50000, 100000, 200000, 300000, 400000, 500000, 600000, 700000, np.Inf]
disc = ArbitraryDiscretiser(binning_dict={'SalePrice': saleprice_map})
df_parallel = disc.fit_transform(df_eda)
df_parallel.head()

In [ ]:
disc.binner_dict_['SalePrice']

NOTE

Create a map to replace the `SalePrice` variable with more informative levels.

In [ ]:
n_classes = len(saleprice_map) - 1
classes_ranges = disc.binner_dict_['SalePrice'][1:-1]

labels_map = {}
for n in range(0, n_classes):
    if n == 0:
        labels_map[n] = f"<{classes_ranges[0]}"
    elif n == n_classes-1:
        labels_map[n] = f"+{classes_ranges[-1]}"
    else:
        labels_map[n] = f"{classes_ranges[n-1]} to {classes_ranges[n]}"

labels_map


Replace using `.replace()`

In [ ]:
df_parallel['SalePrice'] = df_parallel['SalePrice'].replace(labels_map)
df_parallel.head()

Creates multi-dimensional categorical data plot

In [ ]:
import plotly.express as px
fig = px.parallel_categories(df_parallel, color="SalePrice")
fig.show()

* You may add as many sections as you want, as long as they support your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* If you do not need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create here your folder
  # os.makedirs(name='')
except Exception as e:
  print(e)
